# EEG Emotion detection from alpha and beta band entropies on simplified dataset


In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Here is how we extract the features and the labels from the original 55 MB dataset
import csv
emotion_data = pd.read_csv("emotions.csv")   
emotion_data.info()
# give the head only for labels
emotion_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2132 entries, 0 to 2131
Columns: 2549 entries, # mean_0_a to label
dtypes: float64(2548), object(1)
memory usage: 41.5+ MB


,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,...,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,...,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,...,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,...,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,...,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,NEUTRAL


In [ ]:
columns_to_extract = [col for col in emotion_data.columns if 'entropy' in col.lower()] + list(emotion_data.columns[-1:])
filtered_data = emotion_data[columns_to_extract]
filtered_data.to_csv('entropy.csv', index=False)

# Below we directly load the filtered dataset

# Load the data

In [3]:
import csv
emotion_data = pd.read_csv("entropy.csv")   

In [63]:
emotion_data

,entropy0_a,entropy1_a,entropy2_a,entropy3_a,entropy4_a,entropy0_b,entropy1_b,entropy2_b,entropy3_b,entropy4_b
0,0.00,4.98,0.00,0.00,0.0,0.00,4.98,0.00,0.00,0.0
1,4.98,5.00,0.00,4.98,0.0,4.97,5.00,4.97,4.98,0.0
2,0.00,5.00,4.93,0.00,0.0,0.00,5.01,0.00,0.00,0.0
3,0.00,5.01,0.00,0.00,0.0,0.00,5.00,0.00,0.00,0.0
4,0.00,4.99,0.00,0.00,0.0,0.00,4.96,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...
2127,4.99,5.00,4.95,4.98,0.0,4.98,5.00,0.00,4.96,0.0
2128,0.00,5.00,0.00,0.00,0.0,0.00,5.00,0.00,0.00,0.0
2129,0.00,0.00,0.00,0.00,0.0,0.00,4.99,0.00,0.00,0.0
2130,0.00,4.96,0.00,0.00,0.0,0.00,4.96,0.00,0.00,0.0


In [4]:
# LABELS to classes mapping
maps = dict({"NEGATIVE":2,"POSITIVE":1,"NEUTRAL":0})


# Split data into training and testing

In [22]:
from sklearn.preprocessing import OneHotEncoder
# We want to predict the emotion based on the entropy features
y = pd.get_dummies(emotion_data['label'].apply(lambda x: maps[x]))
X = emotion_data.loc[:, 'entropy0_a':'entropy4_b']
# save the current emotion data as entorpy.csv
X.to_csv('entropy.csv', index=False)
X = emotion_data.drop('label', axis=1).copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [23]:
import pandas as pd
from sklearn import preprocessing

x = X_train #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

# Neural Network

In [24]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras


model = models.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation="softmax"))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print(model.summary())

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 256)            │         2,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,099 (141.01 KB)

 Trainable params: 36,099 (141.01 KB)

 Non-trainable params: 0 (0.00 B)

None


In [25]:
model.fit(df,y_train,epochs=30,batch_size=32)

Epoch 1/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5830 - loss: 0.8837 
Epoch 2/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7130 - loss: 0.6285
Epoch 3/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6981 - loss: 0.5919
Epoch 4/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7123 - loss: 0.6072
Epoch 5/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7026 - loss: 0.5993
Epoch 6/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7352 - loss: 0.5630
Epoch 7/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7083 - loss: 0.5829
Epoch 8/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7192 - loss: 0.5623
Epoch 9/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7088 - loss: 0.5884
Epoch 10/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7143 - loss: 0.6083
Epoch 11/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6797 - loss: 0.5541
Epoch 12/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7306 - l

In [27]:
import pandas as pd
from sklearn import preprocessing

x = X_test #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

model.evaluate(df,y_test)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - accuracy: 0.6700 - loss: 0.5967


[0.6222611665725708, 0.6499999761581421]

In [12]:
# CNN

model = models.Sequential()
model.add(layers.Conv1D(8, 4, activation='relu',input_shape=(8,1)))
model.add(layers.Conv1D(4, 2, activation='relu'))
model.add(layers.MaxPool1D(2))
model.add(layers.Flatten())

model.add(layers.Dense(100,activation="relu"))
model.add(layers.Dense(3,activation="softmax"))

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

/opt/homebrew/Caskroom/miniforge/base/envs/TF211/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 5, 8)           │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 4, 4)           │            68 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 2, 4)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 100)            │           900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,311 (5.12 KB)

 Trainable params: 1,311 (5.12 KB)

 Non-trainable params: 0 (0.00 B)

None


In [14]:
# Reshape X_train to match the input shape expected by the model
X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))

model.fit(X_train_reshaped, y_train, epochs=32, batch_size=32)

Epoch 1/32


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_5" is incompatible with the layer: expected axis -1 of input shape to have value 8, but received input with shape (None, 12)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 10, 1), dtype=float32)
  • training=True
  • mask=None